In [5]:
from gazebo_msgs.srv import SpawnModel,SetModelState
import rospy
from geometry_msgs.msg import Pose, TransformStamped
from gazebo_msgs.msg import ModelState 
from random import randrange
import PyKDL
import numpy as np
import time
import tf
import tf2_ros


def Summon_model(model_X,model_Y,model_Z,model_R,model_P,model_Yaw,model_name):

    rot_quat = PyKDL.Rotation.RPY(np.radians(model_R), np.radians(model_P), np.radians(model_Yaw)).GetQuaternion() #0.328462 0 0.000036 0 -1.570604 3.141059
    model_pose = Pose()
    model_pose.position.x = model_X
    model_pose.position.y = model_Y
    model_pose.position.z = model_Z
    model_pose.orientation.x = rot_quat[0]
    model_pose.orientation.y = rot_quat[1]
    model_pose.orientation.z = rot_quat[2]
    model_pose.orientation.w = rot_quat[3]

    spawn_model_client = rospy.ServiceProxy('gazebo/spawn_sdf_model', SpawnModel)
    
    if "cylinder" in model_name:
        print('Summoning Cylinder')
        file_name = 'cylinder'
    elif "box" in model_name:
        print('Summoning Box')
        file_name = 'box'
    elif "sphere" in model_name:
        print('Summoning Sphere')
        file_name = 'sphere'
    elif "Aerofoil" in model_name:
        print('Summoning Aerofoil')
        file_name = 'Aerofoil'
    elif "Carbon_hood_scoop" in model_name:
        print('Summoning Carbon_hood_scoop')
        file_name = 'Carbon_hood_scoop'
    elif "submarine" in model_name:
        print('Summoning submarine')
        file_name = 'submarine'
    elif "Satellite" in model_name:
        print('Summoning Satellite')
        file_name = 'Satellite'        
    elif "Subaru" in model_name:
        print('Summoning Subaru')
        file_name = 'Subaru'
    elif "roller_empty" in model_name:
        print('Summoning roller_empty')
        file_name = 'roller_empty'
    elif "roller_tunnel" in model_name:
        print('Summoning roller_tunnel')
        file_name = 'roller_tunnel'
    elif "pyramids" in model_name:
        print('Summoning pyramids')
        file_name = 'pyramids'
    elif "waves" in model_name:
        print('Summoning waves')
        file_name = 'waves'
    elif "weird" in model_name:
        print('Summoning weird')
        file_name = 'weird'

    stat=spawn_model_client( model_name = model_name, model_xml=open('/home/shadowd/ws_panda/src/sdf_models/' + file_name + '_model.sdf', 'r').read(),
        robot_namespace='', initial_pose=model_pose, reference_frame='world')
    if stat.success == False:
        stat=update_model_pose(1, model_name, [model_X,model_Y,model_Z,model_R,model_P,model_Yaw])
    
    return stat

def update_model_pose(hold_time, model_name, Task_pose):
    
    model_state_msg = ModelState()
    t_end = time.time() + hold_time

    while True:
        model_state_msg.model_name = model_name
        model_state_msg.pose.position.x = Task_pose[0]
        model_state_msg.pose.position.y = Task_pose[1] 
        model_state_msg.pose.position.z = Task_pose[2]

        model_rot_quat = PyKDL.Rotation.RPY(np.radians(Task_pose[3]), np.radians(Task_pose[4]), np.radians(Task_pose[5])).GetQuaternion()
        model_state_msg.pose.orientation.x = model_rot_quat[0]
        model_state_msg.pose.orientation.y = model_rot_quat[1]
        model_state_msg.pose.orientation.z = model_rot_quat[2]
        model_state_msg.pose.orientation.w = model_rot_quat[3]
        resp = set_state( model_state_msg )
        if(time.time() > t_end):
            break
    return resp 


def publish_tf(model_pose, parent_frame, child_frame):
    static_transformStamped.header.stamp = rospy.Time.now()
    static_transformStamped.header.frame_id = parent_frame
    static_transformStamped.child_frame_id = child_frame

    static_transformStamped.transform.translation.x = model_pose[0]
    static_transformStamped.transform.translation.y = model_pose[1]
    static_transformStamped.transform.translation.z = model_pose[2]

    quat = tf.transformations.quaternion_from_euler(np.radians(model_pose[3]),np.radians(model_pose[4]),np.radians(model_pose[5]))
    static_transformStamped.transform.rotation.x = quat[0]
    static_transformStamped.transform.rotation.y = quat[1]
    static_transformStamped.transform.rotation.z = quat[2]
    static_transformStamped.transform.rotation.w = quat[3]
    broadcaster.sendTransform(static_transformStamped)



rospy.init_node('insert_object',log_level=rospy.INFO)
rospy.wait_for_service('/gazebo/spawn_sdf_model')
set_state = rospy.ServiceProxy('/gazebo/set_model_state', SetModelState)

broadcaster = tf2_ros.StaticTransformBroadcaster()
static_transformStamped = TransformStamped()
print("Loaded!")

Loaded!


### FOR STL models

In [6]:
#model_type = ['Aerofoil_'+ str(101), 0.6, 0, 0.15, 0, 0, 0]
#model_type = ['Carbon_hood_scoop_'+ str(101), 0.6, 0, 0.15, 0, 0, 0]

#model_type = ['submarine_'+ str(101), 0.55, 0, 0.0, 0, 0, 90]
#model_type = ['Satellite_'+ str(101), 0.6, -0.8, 0.15, 0, 0, -90]
#model_type = ['roller_empty_'+ str(103),0.4, -0.1, 0.03, 0, 0, 0]
model_type = ['roller_tunnel_'+ str(103),0.1, 0, 0, 0, 0, 90]
#model_type = ['Subaru_'+ str(101), 0.5, 0, 0.15, 0, 0, 0]


box_name = model_type[0]
box_X = model_type[1] #0.6
#box_X = 0.5
box_Y = model_type[2] #0
box_Z = model_type[3] #0.15

box_R = model_type[4] #0
box_P = model_type[5] #0
box_Yaw = model_type[6] #0

Summon_model(box_X,box_Y,box_Z,box_R,box_P,box_Yaw,box_name)

Summoning roller_tunnel


success: True
status_message: "SpawnModel: Successfully spawned entity"

## FOR BOX

In [ ]:
box_name = 'Test_box_'+ str(101)
box_X = 0.3
box_Y = 0.0
box_Z = 0.1

box_R = 0
box_P = 0
box_Yaw = 0

Summon_model(box_X,box_Y,box_Z,box_R,box_P,box_Yaw,box_name)

### Update Box Pose

In [ ]:
Box_pose = [0.3, 0.0, 0.1, 45., 45., 0.]

parent_frame = "world"
child_frame = "gt_plane"
result = update_model_pose(1, box_name, Box_pose)
publish_tf(Box_pose, parent_frame, child_frame)
result

## FOR CYLINDER

In [ ]:
model_name = 'Test_cylinder_'+ str(101)
model_X = 0.3
model_Y = 0.0
model_Z = 0.1

model_R = 90
model_P = 0
model_Yaw = 90

Summon_model(model_X,model_Y,model_Z,model_R,model_P,model_Yaw,model_name)

### Update Cylinder Pose

In [ ]:
Cylinder_pose = [0.3, 0.0, 0.1, 90., 0., 90.]

parent_frame = "world"
child_frame = "gt_cylinder"
result = update_model_pose(1, model_name, Cylinder_pose)
publish_tf(Cylinder_pose, parent_frame, child_frame)
result

## FOR SPHERE

In [ ]:
model_name = 'Test_sphere_'+ str(101)
model_X = 0.3
model_Y = 0.0
model_Z = 0.05

model_R = 90
model_P = 0
model_Yaw = 90

Summon_model(model_X,model_Y,model_Z,model_R,model_P,model_Yaw,model_name)

### Update Sphere Pose

In [ ]:
Capsule_pose = [0.3, 0.0, 0.05, 0., 0., 0.]

parent_frame = "world"
child_frame = "gt_sphere"
result = update_model_pose(1, model_name, Capsule_pose)
publish_tf(Capsule_pose, parent_frame, child_frame)
result